![Insper](https://github.com/danielscarvalho/Insper-DS-Dicas/blob/master/Insper-Logo.png?raw=true)

# Insper Pós-Graduação
## Programa Avançado em Data Science e Decisão [»](https://www.insper.edu.br/pos-graduacao/programas-avancados/programa-avancado-em-data-science-e-decisao/)


# Atividade Integradora
## Setup

### Dependências

In [1]:
import pandas as pd
import numpy as np
from dfply import *
import altair as alt
import missingno as msno

### Carregamento dos Dados

In [2]:
data = pd.read_csv("cs_bisnode_panel.csv")
dicionario_de_dados_0 = pd.read_excel("bisnode_variable_names.xls", header=4)

data.head()

## Dicionário de Dados
### Limpeza

Ao carregar o dicionário de dados a primeira coluna pega seu nome da quarta linha da tabela (argumento `header=4` acima). As outras três colunas são nomeadas abaixo.

In [3]:
dicionario_de_dados_1 = dicionario_de_dados_0.rename({'Unnamed: 1': 'description',
                                                      'Unnamed: 2': 'type',
                                                      'Unnamed: 3': 'footnote'},
                                                     axis=1)

In [ ]:
dicionario_de_dados_1.columns

Então retiramos as linhas não relevantes para a analize, incluindo linhas totalmente em branco e uma linha com informação de versão da base de dados: 
 - `v 0.92. 2021-02-04`

In [4]:
dicionario_de_dados = dicionario_de_dados_1\
                       .drop(index=54)\
                       .dropna(how="all")\
                       .reset_index()\
                       .drop('index', axis='columns')

In [ ]:
dicionario_de_dados.sample(5)

## Dados
### Removendo colunas específicas:

In [5]:
columns_to_remove = ['COGS', 'finished_prod', 'net_dom_sales', 'net_exp_sales', 'wages', 'D']

data.drop(columns=columns_to_remove, inplace=True)

data.columns

Index(['comp_id', 'begin', 'end', 'amort', 'curr_assets', 'curr_liab',
       'extra_exp', 'extra_inc', 'extra_profit_loss', 'fixed_assets',
       'inc_bef_tax', 'intang_assets', 'inventories', 'liq_assets',
       'material_exp', 'personnel_exp', 'profit_loss_year', 'sales',
       'share_eq', 'subscribed_cap', 'tang_assets', 'balsheet_flag',
       'balsheet_length', 'balsheet_notfullyear', 'year', 'founded_year',
       'exit_year', 'ceo_count', 'foreign', 'female', 'birth_year',
       'inoffice_days', 'gender', 'origin', 'nace_main', 'ind2', 'ind',
       'urban_m', 'region_m', 'founded_date', 'exit_date', 'labor_avg'],
      dtype='object')

### Removendo dados do ano 2016:

Registros que começaram ou terminaram (ocorreram em) 2016 são removidos do conjunto.

In [6]:
colunas_data = ["begin", "end", "founded_date", "exit_date"]

data.dtypes[colunas_data]

begin           object
end             object
founded_date    object
exit_date       object
dtype: object

Dados destas colunas precisam estar em formato de datetime para serem operados corretamente, então vamos converte-los.

In [7]:
for column in colunas_data:
    data[column] = pd.to_datetime(data[column], format="%Y-%m-%d")

data[colunas_data].dtypes

begin           datetime64[ns]
end             datetime64[ns]
founded_date    datetime64[ns]
exit_date       datetime64[ns]
dtype: object

Removendo Registros do ano de 2016:

In [8]:
data = data[data["year"]!=2016]
data["year"].unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

### Criando coluna para Variável Dependente:
---

- Vamos operar com o conceito de "atividade", empresas que não tiveram atividade por mais de 2 anos são consideradas "inativas".

Obs.: ativa = 0, inativa = 1

In [32]:
# Sort the DataFrame by company and year
data.sort_values(by=['comp_id', 'year'], inplace=True)


# Create shifted columns to check sales in the following 2 years
conditions_x1 = [((data['comp_id'] == data['comp_id'].shift(-1)) &
                  (data['year'] == data['year'].shift(-1) - 1)),

                 ((data['comp_id'] != data['comp_id'].shift(-1)) |
                  data['year'] != data['year'].shift(-1))]

values_x1 = [data['sales'].shift(-1),
             np.nan]

data['sales_x1'] = np.select(conditions_x1, values_x1)


conditions_x2 = [
    ((data['comp_id'] == data['comp_id'].shift(-1)) &
     (data['year'] == data['year'].shift(-1) - 2)),

    ((data['comp_id'] == data['comp_id'].shift(-2)) &
     (data['year'] == data['year'].shift(-2) - 2)),

    True
]

values_x2 = [data['sales'].shift(-1),
             data['sales'].shift(-2),
             np.nan]

data['sales_x2'] = np.select(conditions_x2, values_x2)


# data.loc[data['comp_id'] == data['comp_id'].shift(-1), 'sales_x1'] = data['sales'].shift(-1)
# data.loc[data['comp_id'] == data['comp_id'].shift(-2), 'sales_x2'] = data['sales'].shift(-2)

# Create a condition to identify companies that ceased to operate 
# (sem vendas por mais de 2 anos)
condition = (data['sales_x1'] == 0) & (data['sales_x2'] == 0)

# Create a new 'dependente' column with 1 for ceased companies and 0 otherwise
data['dependente'] = condition.astype(int)


Vamos observar as duas primeiras empresas da dataframe para averiguar seus estados de "dependencia":

In [ ]:
# lista de colunas para avaliação:
check_list=["comp_id", "year", "sales", "sales_x1", "sales_x2", "dependente"]

data[check_list].head(18)

Vamos averiguar agora os anos em que as empresas tiveram atividade e inatividade:

In [37]:
data_grouped = data.groupby('comp_id')

#Contando anos de acompanhamento
comp_years = data_grouped['year'].count()

#Contando anos de "atividade"
active_years = data_grouped.apply(lambda group: (group['sales'] > 0).sum())

#contando anos de "inatividade"
inactive_years = data_grouped.apply(lambda group: (group['sales'] == 0).sum())

#Contando anos de "dependência"
closed_years = data_grouped.apply(lambda group: (group['dependente'] == 1).sum())

In [38]:
activity_df = pd.DataFrame({"Total years":comp_years,
                            "Active years":active_years,
                            "Inactive years":inactive_years,
                            "Closed years":closed_years})

activity_df

,Total years,Active years,Inactive years,Closed years
comp_id,,,,
1.001034e+06,11,6,5,4
1.001541e+06,7,5,2,1
1.002029e+06,9,9,0,0
1.003200e+06,8,8,0,0
1.004012e+06,8,8,0,0
...,...,...,...,...
4.640446e+11,8,7,1,0
4.640568e+11,3,3,0,0
4.640793e+11,1,0,1,0


## 2012

In [43]:
data_2012 = data[data['year'] == 2012]
data_2012.year.unique()

array([2012])

In [48]:
import math

### Sales

In [44]:
data_2012.sales.describe()

count    2.744600e+04
mean     4.364306e+05
std      3.531813e+06
min     -1.834445e+04
25%      3.248148e+03
50%      2.515556e+04
75%      9.676389e+04
max      1.058662e+08
Name: sales, dtype: float64

Valores negativos não são válidos para a váriavel `sales`. O tratamento escolhido nesse caso é substituir valores negativos por 0.

In [45]:
data_2012.loc[data['sales'] < 0] = 0
data_2012.sales.describe()

count    2.744600e+04
mean     4.364316e+05
std      3.531813e+06
min      0.000000e+00
25%      3.248148e+03
50%      2.515556e+04
75%      9.676389e+04
max      1.058662e+08
Name: sales, dtype: float64

Podemos ver ainda nas estatísticas descritivas que a média é maior que o terceiro quartil, indicando uma distribuição bastante assimétrica. Criamos entao uma com o logarítmo de `sales` para auxiliar na análise.

In [79]:
data_2012.loc[:,'log_sales'] = data_2012.sales\
                                        .apply(lambda x: math.log(x)\
                                               if x != 0\
                                               else 0)

data_2012.log_sales.describe()

count    27446.000000
mean         8.803959
std          4.370550
min          0.000000
25%          8.085840
50%         10.132834
75%         11.480029
max         18.477687
Name: log_sales, dtype: float64